In [29]:
# Dependencies to pull API
import requests
import json
# Google developer API key
from config import gkey

In [32]:
# Import Dependencies for Database
from config import password
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
import psycopg2
import pandas as pd

In [31]:
db_string = f"postgresql+psycopg2://postgres:" + password + "@127.0.0.1:5434/WineEnthusiast"

In [11]:
engine = create_engine(db_string)
inspector = inspect(engine)
inspector.get_table_names()

['winemag', 'wineregions', 'wine', 'uswine', 'winedata', 'winedata_ml']

In [5]:
connection = engine.connect
session = Session(engine)
engine.execute("SELECT * from wine")

In [6]:
col_names_list = []

for i in range(len(inspector.get_columns('wine'))):
    col_names_list.append(inspector.get_columns('wine')[i]['name'])
    
print(col_names_list)

['index', 'description', 'points', 'price', 'province', 'region_1', 'region_2', 'title', 'variety', 'winery']


In [12]:
col_names = inspector.get_columns('wine')[5]['name']
col_names

'region_1'

In [13]:
location_wine_df = pd.DataFrame(columns = col_names_list)

In [14]:
# Inner join for wines and regions
import sys
join_db = engine.execute("SELECT region_1 from wine")
for record in join_db:
    record_series = pd.Series((record), index = location_wine_df.columns)
    
    location_wine_df = location_wine_df.append(record_series, ignore_index=True, verify_integrity = True).drop_duplicates()

In [15]:
location_wine_df.columns

Index(['index', 'description', 'points', 'price', 'province', 'region_1',
       'region_2', 'title', 'variety', 'winery'],
      dtype='object')

In [16]:
region_df = location_wine_df.drop(columns = ['index', 'description', 'points', 'price', 'province',
       'region_2', 'title', 'variety', 'winery'], axis = 1)
region_df

,region_1
0,Willamette Valley
1,Lake Michigan Shore
2,Napa Valley
3,Alexander Valley
4,Central Coast
...,...
261,Monterey County-Napa County
262,San Diego County
263,Niagara Escarpment
264,Sonoma-Napa-Lake


In [43]:
# Target valley
region = "Lodi"

# Build the endpoint URL
valley_url = ('https://maps.googleapis.com/maps/api/place/findplacefromtext/json?'
    'input={0}&key={1}&inputtype=textquery').format(region, gkey)

In [44]:
# Run a request to endpoint and convert result to json
geo_data = requests.get(valley_url).json()

# Print the json
print(geo_data)

{'candidates': [{'place_id': 'ChIJNTjFJtmhmoARRuhsu5jkrY4'}], 'status': 'OK'}


In [45]:
place_id = geo_data['candidates'][0]['place_id']
place_id

'ChIJNTjFJtmhmoARRuhsu5jkrY4'

In [46]:
place_url = ('https://maps.googleapis.com/maps/api/place/details/json?place_id=ChIJNTjFJtmhmoARRuhsu5jkrY4&key='+ gkey)

In [47]:
place_data = requests.get(place_url).json()
print(place_data)

{'html_attributions': [], 'result': {'address_components': [{'long_name': 'Lodi', 'short_name': 'Lodi', 'types': ['locality', 'political']}, {'long_name': 'San Joaquin County', 'short_name': 'San Joaquin County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'California', 'short_name': 'CA', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}], 'adr_address': '<span class="locality">Lodi</span>, <span class="region">CA</span>, <span class="country-name">USA</span>', 'formatted_address': 'Lodi, CA, USA', 'geometry': {'location': {'lat': 38.1341477, 'lng': -121.2722194}, 'viewport': {'northeast': {'lat': 38.15494201474051, 'lng': -121.2413109180588}, 'southwest': {'lat': 38.08245401935762, 'lng': -121.4052700246796}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png', 'icon_background_color': '#7B9EB0', 'icon_mask_base_uri': 'https://maps.g

In [48]:
latitude = place_data['result']['geometry']['location']['lat']
latitude


38.1341477

In [49]:
longitude = place_data['result']['geometry']['location']['lng']
longitude

-121.2722194